In [1]:
!pip install geopandas fiona 

     |████████████████████████████████| 972kB 28.3MB/s 
     |████████████████████████████████| 14.8MB 219kB/s 
     |████████████████████████████████| 6.5MB 49.8MB/s 


In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
#Goal here is to read in & access this ~1gb file
#File is in this area (of my drive, at least): /content/drive/My Drive/College Year 3/20F/ENGS108/ENGS108_Final_Project/Data_grip4
# Trying: https://gis.stackexchange.com/questions/32762/how-to-access-feature-classes-in-file-geodatabases-with-python-and-gdal (look far down)

#Data taken from: https://www.globio.info/download-grip-dataset
#Some similar data here: https://enterprisecontentnew-usfs.hub.arcgis.com/datasets/motor-vehicle-use-map-roads-feature-layer?page=17611


import fiona 
import geopandas as gpd

# Get all the layers from the .gdb file 
#%%%%%%%%%%%%%%%%% We May Have Different Drive Paths To This Folder %%%%%%%%%%%%%%%%%%%%
gdb_file_path =  '/content/drive/MyDrive/College Year 3/20F/ENGS108/ENGS108_Final_Project/Data_Sets/GRIP4_region1.gdb'
# Vlado's Path: gdb_file_path = '/content/drive/My Drive/ENGS108_Final_Project/Data_Sets/GRIP4_region1.gdb'


water_bodies_path =  '/content/drive/MyDrive/College Year 3/20F/ENGS108/ENGS108_Final_Project/Data_Sets/hydropolys.gdb'
linear_water_path =  '/content/drive/MyDrive/College Year 3/20F/ENGS108/ENGS108_Final_Project/Data_Sets/hydrolines.gdb'
#water_bodies_path = '/content/drive/My Drive/ENGS108_Final_Project/Data_Sets/hydropolys.gdb'
#linear_water_path = '/content/drive/My Drive/ENGS108_Final_Project/Data_Sets/hydrolines.gdb'


#gdb_file_path = '/content/drive/MyDrive/College Year 3/20F/ENGS108/ENGS108_Final_Project/Data_Sets/GRIP4_region1.gdb'
layers = fiona.listlayers(gdb_file_path)

#for layer in layers:
    #gdf = gpd.read_file(gdb_file_path,layer=layer)
    #print(gdf)
    # Do stuff with the gdf

for layername in fiona.listlayers(gdb_file_path):
    with fiona.open(gdb_file_path, layer=layername) as src:
        print(layername, len(src))
c = fiona.open(gdb_file_path, 'r')
print(len(src))
print(type(src))
print(len(c))
c[10060]

linear_water = fiona.open(linear_water_path, 'r')
poly_water = fiona.open(water_bodies_path, 'r')

#gdf = gpd.read_file(gdb_file_path)
#gdf.head(2)

GRIP4_region1 5360526
5360526
<class 'fiona.collection.Collection'>
5360526


In [4]:
c[6]['geometry']['coordinates']

[[(-123.15683999999999, 49.15667000000008),
  (-123.15675999999996, 49.14998000000003),
  (-123.15675999999996, 49.14206000000007),
  (-123.15674999999999, 49.134930000000054)]]

In [5]:
import datetime
now = datetime.datetime.now()
print(now)

'''
Finds the indeces of c that are within our desired range
'''
def linear_grid_finder(clist,lat_min,lat_max,long_min,long_max, kind):
  idcs = []
  for i in range(1,len(clist)+1):
      if kind!=1:
        #print("latitude:", lat_min, clist[i]['geometry']['coordinates'][0][0][1], lat_max)
        #print("longitude:", long_min, clist[i]['geometry']['coordinates'][0][0][0], long_max)
        if lat_min < clist[i]['geometry']['coordinates'][0][0][1] < lat_max:
          if long_min < clist[i]['geometry']['coordinates'][0][0][0] < long_max:
            idcs.append(i)
      else:
         if lat_min < clist[i]['geometry']['coordinates'][0][0][0][1] < lat_max:
          if long_min < clist[i]['geometry']['coordinates'][0][0][0][0] < long_max:
            idcs.append(i)
  return idcs

now = datetime.datetime.now()
print(now)

2020-12-02 16:26:35.803615
2020-12-02 16:26:35.803902


In [6]:
'''
Splits a rectangular surface into smaller rectangles and returns them all
'''
def split_grid(num_rows,num_cols,x_min,x_max,y_min,y_max):
    quad_list = []
    increment_rows = (y_max - y_min)/num_rows
    increment_cols = (x_max - x_min)/num_cols

    for i in range(1,num_rows):
      for j in range(1,num_cols):
          quad_list.append((x_min+(i-1)*increment_cols, x_min+(i)*increment_cols, y_min+(j-1)*increment_rows, y_min+(j)*increment_rows))
    return quad_list

In [ ]:
print(split_grid(10,10,0,100,0,100))

[(0.0, 10.0, 0.0, 10.0), (0.0, 10.0, 10.0, 20.0), (0.0, 10.0, 20.0, 30.0), (0.0, 10.0, 30.0, 40.0), (0.0, 10.0, 40.0, 50.0), (0.0, 10.0, 50.0, 60.0), (0.0, 10.0, 60.0, 70.0), (0.0, 10.0, 70.0, 80.0), (0.0, 10.0, 80.0, 90.0), (10.0, 20.0, 0.0, 10.0), (10.0, 20.0, 10.0, 20.0), (10.0, 20.0, 20.0, 30.0), (10.0, 20.0, 30.0, 40.0), (10.0, 20.0, 40.0, 50.0), (10.0, 20.0, 50.0, 60.0), (10.0, 20.0, 60.0, 70.0), (10.0, 20.0, 70.0, 80.0), (10.0, 20.0, 80.0, 90.0), (20.0, 30.0, 0.0, 10.0), (20.0, 30.0, 10.0, 20.0), (20.0, 30.0, 20.0, 30.0), (20.0, 30.0, 30.0, 40.0), (20.0, 30.0, 40.0, 50.0), (20.0, 30.0, 50.0, 60.0), (20.0, 30.0, 60.0, 70.0), (20.0, 30.0, 70.0, 80.0), (20.0, 30.0, 80.0, 90.0), (30.0, 40.0, 0.0, 10.0), (30.0, 40.0, 10.0, 20.0), (30.0, 40.0, 20.0, 30.0), (30.0, 40.0, 30.0, 40.0), (30.0, 40.0, 40.0, 50.0), (30.0, 40.0, 50.0, 60.0), (30.0, 40.0, 60.0, 70.0), (30.0, 40.0, 70.0, 80.0), (30.0, 40.0, 80.0, 90.0), (40.0, 50.0, 0.0, 10.0), (40.0, 50.0, 10.0, 20.0), (40.0, 50.0, 20.0, 30.0),

In [7]:
#idcs = linear_grid_finder(c,goal_lat_min,goal_lat_max, goal_long_min, goal_long_max,0)

def return_coords(clist, idx, kind):
  if kind == 1:
     return clist[idx]['geometry']['coordinates'][0][0]
  else:
     return clist[idx]['geometry']['coordinates'][0]

def return_all_coords(clist, idcs, kind=0):
  all_coords=[]
  for idx in idcs:
      if clist[idx]:
        if len(clist[idx]['geometry']['coordinates'])==1:
          all_coords.append(return_coords(clist,idx, kind))
        else:
          for i in range(len(clist[idx]['geometry']['coordinates'])):
              all_coords.append(clist[idx]['geometry']['coordinates'][i])
  return all_coords

In [8]:
!pip install fastkml pykml
from fastkml import kml
from pykml import parser
import re

     |████████████████████████████████| 71kB 8.0MB/s 
     |████████████████████████████████| 51kB 7.4MB/s 
  Created wheel for fastkml: filename=fastkml-0.11-cp36-none-any.whl size=61869 sha256=42cbb1532e86e3bc0a88e50a5708b1159bba2a7bc3f907bb73dc1f25aa469444
  Stored in directory: /root/.cache/pip/wheels/55/01/ea/6191eb73e0894743d02b33a2b1a570e85242844d810804fbf2
  Created wheel for pygeoif: filename=pygeoif-0.7-cp36-none-any.whl size=19249 sha256=665ee31ab21e492303c6a73b9e65b79c016ba65e1eeb2bd62fcce75430819952
  Stored in directory: /root/.cache/pip/wheels/60/6e/a7/3d3eef59ac84a86663d0f5c5a92091f5056e9aeb6588c4de34
Successfully built fastkml pygeoif


In [10]:
###GEO Processing###
NYC_data_path = "/content/drive/MyDrive/College Year 3/20F/ENGS108/ENGS108_Final_Project/Data_Sets/boston_contours2.kml"

def process_geo(geo_data_path):
  root = parser.fromstring(open(NYC_data_path, 'r').read())
  sample = []
  for i in range(0, len(root.Document.Placemark)):
    unprocessed_string = str(root.Document.Placemark[i].LineString.coordinates)
    items = re.split('[,|\n]', unprocessed_string) #uses regex to split by "," or "\n"
    del items[2::3] #delete zeros
    del items[-1] #delete last item
    processed_string = []
    for i in range(0,len(items),2):
      coord_tup = (float(items[i]), float(items[i+1]))
      processed_string.append(coord_tup)

    sample.append(processed_string)
  return sample #Returns list of graphable coordinates


In [11]:
###GEO Processing Contd###
def linear_grid_finder_geo(geo_object_list,lat_min,lat_max,long_min,long_max, kind = 0):
  idcs = []
  for i in range(1,len(geo_object_list)):
    #print("latitude",lat_min > geo_object_list[i][0][1] > lat_max)
    #print("latitude",lat_min,  geo_object_list[i][0][1], lat_max)
    #print("longitude", long_min < geo_object_list[i][0][0] < long_max)
    #print("longitude", long_min , geo_object_list[i][0][0], long_max)
    if lat_min < geo_object_list[i][0][1] < lat_max:
      if long_min < geo_object_list[i][0][0] < long_max:
        idcs.append(i)

  return idcs


def return_coords_geo(geo_object_list, idx, kind):
    return geo_object_list[idx]


def return_all_coords_geo(geo_object_list, idcs, kind=0):
  all_coords=[]
  for idx in idcs:
    all_coords.append(geo_object_list[idx])
  return all_coords

In [12]:
import matplotlib.pyplot as plt

def prepare_plot(roads_data):
  prepared_dataX = []
  prepared_dataY = []
  for feature in roads_data:
    these_xs = []
    these_ys = []
    try:
      for x, y in feature:#apparently this is faster than *zip(): https://stackoverflow.com/questions/8081545/how-to-convert-list-of-tuples-to-multiple-lists
        these_xs.append(x)
        these_ys.append(y)
    except:
      for x, y in feature[0]:
        these_xs.append(x)
        these_ys.append(y)

    #print("prepared_dataX receives", these_xs)
    prepared_dataX.append(these_xs)

    #print("prepared_dataY receives", these_ys)
    prepared_dataY.append(these_ys)
  return prepared_dataX, prepared_dataY

#All done. Now we can graph this stuff

In [13]:
from matplotlib.pyplot import figure
from matplotlib import pyplot as plt
plt.ioff()
import numpy as np
###GEO Edit included###
def create_image(roadX, roadY, waterX_poly, waterY_poly, waterX_lin, waterY_lin, geoX, geoY, n):
  fig = figure(num=None, figsize=(10, 10), dpi=100, facecolor='w', edgecolor='k') #Resize to 1000x1000 images
  if roadX:
    for i in range(0, len(roadX)):
      tuple_setX = roadX[i]
      tuple_setY = roadY[i]
      for j in range(0, len(tuple_setX)-1, 1):
        plt.plot(tuple_setX[j:j+2], tuple_setY[j:j+2], '-', linewidth = 2, color = 'grey')
  if waterX_lin:
    for i in range(0, len(waterX_lin)):
      tuple_setX = waterX_lin[i]
      tuple_setY = waterY_lin[i]
      for j in range(0, len(tuple_setX)-1, 1):
        plt.plot(tuple_setX[j:j+2], tuple_setY[j:j+2], '-', linewidth = 8, color = 'blue')
  if geoX:
    for i in range(0, len(geoX)):
      tuple_setX = geoX[i]
      tuple_setY = geoY[i]
      for j in range(0, len(geoX)-1, 1):
        plt.plot(tuple_setX[j:j+2], tuple_setY[j:j+2], '-', linewidth = 1, color = 'black')
  if waterX_poly:
      for i in range(len(waterX_poly)):
          plt.fill(waterX_poly[i], waterY_poly[i], edgecolor='blue', facecolor='blue')
  plt.axis('off')
  #plt.savefig('/content/drive/My Drive/ENGS108_Final_Project/Data_Sets/our_images/roads_' +  str(n) + '.png', bbox_inches = 'tight')
  plt.savefig('/content/drive/MyDrive/College Year 3/20F/ENGS108/ENGS108_Final_Project/Data_Sets/our_images_with_geo/' + 'testABC' + str(n) + '.png', bbox_inches = 'tight')

In [14]:
total=100

In [15]:
goal_lat_min = 42.32374440434734 #these are bounds of the Boston Contours2.kml geoData set. It is the "limiting reagent" in terms of the available data
goal_lat_max = 42.52344946162212
x_cuts = int(abs(goal_lat_max-goal_lat_min)*50)
goal_long_min = -71.41581679714523
goal_long_max = -70.97945357693038
y_cuts = int(abs(goal_long_max-goal_long_min)*50)
list_of_coords = split_grid(x_cuts,y_cuts,goal_lat_min,goal_lat_max, goal_long_min, goal_long_max)
print("# of cubes we are splitting up:", len(list_of_coords))
geo_data = process_geo(NYC_data_path)
for k in range(1,len(list_of_coords)):
    print('Im here')
    min_lat, max_lat, min_long, max_long = list_of_coords[k]
    print('here1')
    idcs = linear_grid_finder(c, min_lat, max_lat, min_long, max_long, 0)
    print('here21')
    print("and we might've found some roads:", len(idcs))
    if len(idcs) > 0:
      total+=1
      print(k)
      all_coords = return_all_coords(c,idcs)
      idcs = linear_grid_finder(poly_water, min_lat, max_lat, min_long, max_long, 1)
      lake_coords = return_all_coords(poly_water, idcs,1)
      idcs = linear_grid_finder(linear_water,  min_lat, max_lat, min_long, max_long, 2)
      river_coords = return_all_coords(linear_water, idcs,2)

      idcs = linear_grid_finder_geo(geo_data, min_lat, max_lat, min_long, max_long, 0)
      geo_coords = return_all_coords_geo(geo_data, idcs,0)




      roadX,roadY = prepare_plot(all_coords)
      wpolyX, wpolyY = prepare_plot(lake_coords)
      wlinX, wlinY = prepare_plot(river_coords)
      geoX, geoY = prepare_plot(geo_coords)

      create_image(roadX,roadY, wpolyX, wpolyY, wlinX, wlinY, geoX, geoY, k+total)

# of cubes we are splitting up: 160


FileNotFoundError: ignored

In [ ]:
len(list_of_coords)

361